Using various tree based approaches in Spark to determine whether a College or University is private or public.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('CollegeTree').getOrCreate()

In [4]:
data = spark.read.csv('../../SparkData/College.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(
inputCols=[
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'
], outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [13]:
output_fixed = indexer.fit(output).transform(output)

In [14]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [15]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [16]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                      GBTClassifier, 
                                      RandomForestClassifier)

In [18]:
from pyspark.ml import pipeline

In [19]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [21]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [22]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
bin_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [25]:
print('DTC')
print(bin_eval.evaluate(dtc_preds))

DTC
0.9186746987951806


In [26]:
print('RFC')
print(bin_eval.evaluate(rfc_preds))

RFC
0.9858699503897943


In [28]:
bin_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',
                                         rawPredictionCol='prediction')

In [29]:
print('GBT')
print(bin_eval2.evaluate(gbt_preds))

GBT
0.8539156626506024


In [30]:
rfc_model.featureImportances

SparseVector(17, {0: 0.0214, 1: 0.0438, 2: 0.1438, 3: 0.0111, 4: 0.0068, 5: 0.2003, 6: 0.0624, 7: 0.2561, 8: 0.0356, 9: 0.0088, 10: 0.0087, 11: 0.0088, 12: 0.0103, 13: 0.0931, 14: 0.0252, 15: 0.0473, 16: 0.0167})